In [1]:
import os

In [2]:
%pwd

'k:\\Environments\\End-End_ML_pipeline\\research'

In [3]:
os.chdir("../")

In [11]:
import dagshub
dagshub.init(repo_owner='KerithPaul', repo_name='End-End_ML_pipeline', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

[2025-05-20 22:14:03,126]: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as KerithPaul

[2025-05-20 22:14:03,135]: INFO: helpers: Accessing as KerithPaul]
[2025-05-20 22:14:03,828]: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/KerithPaul/End-End_ML_pipeline "HTTP/1.1 200 OK"]
[2025-05-20 22:14:04,415]: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "KerithPaul/End-End_ML_pipeline"

[2025-05-20 22:14:04,420]: INFO: helpers: Initialized MLflow to track repo "KerithPaul/End-End_ML_pipeline"]


Repository KerithPaul/End-End_ML_pipeline initialized!

[2025-05-20 22:14:04,422]: INFO: helpers: Repository KerithPaul/End-End_ML_pipeline initialized!]
🏃 View run caring-koi-760 at: https://dagshub.com/KerithPaul/End-End_ML_pipeline.mlflow/#/experiments/0/runs/93031df15b4d4897ada575da8e865a6d
🧪 View experiment at: https://dagshub.com/KerithPaul/End-End_ML_pipeline.mlflow/#/experiments/0


In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
   root_dir: Path
   test_data_path: Path
   model_path: Path
   metric_file_name: Path
   all_params: dict
   target_column: str
   mlflow_uri: str

In [19]:
from DS_Ops.constants import *
from DS_Ops.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
            config = self.config.model_evaluation
            params = self.params.ElasticNet
            schema = self.schema.TARGET_COLUMN

            create_directories([config.root_dir])

            model_evaluation_config = ModelEvaluationConfig(
                root_dir = config.root_dir,
                test_data_path = config.test_data_path,
                model_path = config.model_path,
                all_params = params,
                metric_file_name= config.metric_file_name,
                target_column = schema.name,
                mlflow_uri = "https://dagshub.com/KerithPaul/End-End_ML_pipeline.mlflow"
            )
            return model_evaluation_config

In [21]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [22]:
import mlflow.sklearn
from DS_Ops.utils.common import save_json


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        
        return rmse, mae, r2
    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path = Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            if tracking_url_type_store != "file":

                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")

            else:

                mlflow.sklearn.log_model(model, "model")

                

In [23]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()

except Exception as e:
    raise e

[2025-05-20 22:17:42,461]: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-20 22:17:42,474]: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-20 22:17:42,476]: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-20 22:17:42,477]: INFO: common: created directory at: ./artifacts]
[2025-05-20 22:17:42,479]: INFO: common: created directory at: artifacts/model_evaluation]
[2025-05-20 22:17:42,875]: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/05/20 22:17:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/05/20 22:17:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.


🏃 View run powerful-jay-36 at: https://dagshub.com/KerithPaul/End-End_ML_pipeline.mlflow/#/experiments/0/runs/af83a634d63f4a70bdc4db2e973221c4
🧪 View experiment at: https://dagshub.com/KerithPaul/End-End_ML_pipeline.mlflow/#/experiments/0
